# The Guardian API

In the `05_web_scraping_beautiful_soup.ipynb` notebook, we saw examples on how BeautifulSoup can be used 
to parse messy HTML, to extract information, and to act as a rudimentary web crawler. 
We used The Guardian as an illustrative example about how this can be achieved. 
The reason for choosing The Guardian was because they provide a REST API to their servers. 
With the REST API it is possible to perform specific queries on their servers, and to receive 
current information from their servers according to their API guide (ie in JSON)

http://open-platform.theguardian.com/

In order to use their API, you will need to register for an API key. 
At the time of writing (Jan 28, 2020) this was an automated process that can be completed at 

https://bonobo.capi.gutools.co.uk/register/developer

On registration you will receive an API key which will look like: 303qwe2k-xxxx-xxxx-xxxx-eff86a248059

The API is documented here: 

http://open-platform.theguardian.com/documentation/

and Python bindings to their API are provided by The Guardian here

https://github.com/prabhath6/theguardian-api-python

and these can easily be integrated into a web-crawler based on API calls, rather than being based 
on HTML parsing, etc. 

We use four parameters in our queries here: 

1. `section`: the section of the newspaper that we are interested in querying. In this case we will look at 
the technology section 

2. `order-by`: We have specified that the newest items should be closer to the front of the query list 

3. `api-key`: In this notebook, the api-key is left as `test` (works here), but for *real* deployment of such a spider an API key obtained from Guardian should be specified. For the lab tasks, you should replace `test` API key with your personal API key. 

4. `page-size`: The number of results to return. 

In [1]:
from __future__ import print_function

import requests 
import json 

Hard coding a secret such as a password or a key in a code file is a security smell. Instead of hard coding your api-key, you should store your api-key in a config file.

# Fetch Api-Key from Config File 
#### Note: You need to create a config file with your API your api-key and place it under the same directory as this notebook, otherwise the code block below will not work 

The content on my config file looks like below: 

```
[guardian]
api-key=303qwe2k-xxxx-xxxx-xxxx-eff86a248059
```

My config file is named: ```myconfig.cfg```

Note: The api-key above is not a real Guardian api-key. If you use that, you will get an error. 

In [2]:
from configparser import ConfigParser

parser = ConfigParser()
parser.read('./myconfig.cfg')

myapikey = parser.get('guardian', 'api-key')

# If you cannot create a config file, comment the three lines above and uncomment the line below.

# myapikey = '303qwe2k-xxxx-xxxx-xxxx-eff86a248059' # Replace with your api-key. This is not a real api-key.
# print(myapikey)

# Inspect all sections and search for technology-based sections

In [3]:
url = 'https://content.guardianapis.com/sections?api-key=' + myapikey
req = requests.get(url)
src = req.text 

In [4]:
json.loads(src)['response']['status']

'ok'

In [5]:
sections = json.loads(src)['response']

print(sections.keys())

dict_keys(['status', 'userTier', 'total', 'results'])


In [6]:
print(json.dumps(sections['results'][0], indent=2, sort_keys=True))

{
  "apiUrl": "https://content.guardianapis.com/about",
  "editions": [
    {
      "apiUrl": "https://content.guardianapis.com/about",
      "code": "default",
      "id": "about",
      "webTitle": "About",
      "webUrl": "https://www.theguardian.com/about"
    }
  ],
  "id": "about",
  "webTitle": "About",
  "webUrl": "https://www.theguardian.com/about"
}


In [7]:
for result in sections['results']: 
    if 'tech' in result['id'].lower(): 
        print(result['webTitle'], result['apiUrl'])

Technology https://content.guardianapis.com/technology


# Manual query on whole API

In [8]:
# Specify the arguments
args = {
    'section': 'technology', 
    'order-by': 'newest', 
    'api-key': myapikey, 
    'page-size': '100',
#     'q' : 'privacy%20AND%20data'
}

# Construct the URL
base_url = 'http://content.guardianapis.com/search'
url = '{}?{}'.format(
    base_url, 
    '&'.join(["{}={}".format(kk, vv) for kk, vv in args.items()])
)

# Make the request and extract the source
req = requests.get(url) 
src = req.text

In [9]:
print('Number of byes received:', len(src))

Number of byes received: 59882


The API returns JSON, so we parse this using the in-built JSON library. 
The API specifies that all data are returned within the `response` key, even under failure. 
Thereofre, I have immediately descended to the response field 

# Parsing the JSON

In [10]:
response = json.loads(src)['response']
print('The following are available:\n ', sorted(response.keys()))

The following are available:
  ['currentPage', 'orderBy', 'pageSize', 'pages', 'results', 'startIndex', 'status', 'total', 'userTier']


# Verifying the status code

It is important to verify that the status message is `ok` before continuing - if it is not `ok` no 'real' data 
will have been received. 

In [11]:
assert response['status'] == 'ok'

# Listing the results 

The API standard states that the results will be found in the `results` field under the `response` field. 
Furthermore, the URLs will be found in the `webUrl` field, and the title will be found in the `webTitle` 
field. 

First let's look to see what a single result looks like in full, and then I will print a restricted 
set of parameters on the full set of results .

In [12]:
print(json.dumps(response['results'][0], indent=2, sort_keys=True))

{
  "apiUrl": "https://content.guardianapis.com/technology/2021/feb/07/are-share-trading-apps-a-safe-way-to-play-the-markets",
  "id": "technology/2021/feb/07/are-share-trading-apps-a-safe-way-to-play-the-markets",
  "isHosted": false,
  "pillarId": "pillar/news",
  "pillarName": "News",
  "sectionId": "technology",
  "sectionName": "Technology",
  "type": "article",
  "webPublicationDate": "2021-02-07T13:00:11Z",
  "webTitle": "Are share-trading apps a safe way to play the markets?",
  "webUrl": "https://www.theguardian.com/technology/2021/feb/07/are-share-trading-apps-a-safe-way-to-play-the-markets"
}


## Task 6. Response Statistics  

Use Guardian's API to identify the count of all news stories published under the Technology section. List the page size and the number of pages in the result set.

Note that I commented out ```'q' : 'privacy%20AND%20data'``` in the args few blocks above.

In [13]:
# Solution to Task 6

print('Total news stories:', response['total']) 
print('Pages:', response['pages'])
print('Page size:', response['pageSize'])

Total news stories: 53939
Pages: 540
Page size: 100


## Task 7. News Stories About a Specific Topic 

Return all stories in the technology section that are about privacy.

Solution: ```'q' : 'privacy'```

Filter the stories that talk about WhatsApp and Signal.  

Solution: ```'q' : 'privacy%20AND%20whatsapp%20AND%20signal'```


Are there any privacy stories talking about privacy, Whatsapp, and Signal that do not talk about AI? 

Solution: ```'q' : 'privacy%20AND%20whatsapp%20AND%20signal'```


List these stories. Solution in the code cell below. 


Other search queries 7a and 7b are similar to Task 7. I list the attributes below. 

#### 7a. All News Stories About a Phrase 

Return all news stories that are about stock squeeze. 

```'q' : '%22short%20squeeze%22'```

List the ones that are in the business section of the Guardian. 

```
'q' : '%22short%20squeeze%22'
'section' : 'business'
```


#### 7b. All News Stories About a Person 

Return all news stories about Elon Musk published by Guardian in 2020 and 2021.  

How many of these news stories are about bitcoin?  Of the stories that are about Elon Musk and Bitcoin, how many of those do not mention Tesla?  

```
'from-date':'2020-01-01'
'to-date':'2021-02-03'
'order-by':'newest'
'page:1'
'q':'%22Elon%20Musk%22%20AND%20bitcoin%20AND%20NOT%20Tesla' 
```
\# q = "Elon Musk" AND bitcoin AND NOT Tesla

Hints. In the search string, you could use ```%20``` for ```space```; ```%22``` for ```double quote```; ```AND``` for conjunction; ```OR``` for disjunction; and ```NOT``` for negation

In [14]:
# Solution to Task 7 below  

args = {
    'section': 'technology', 
    'order-by': 'newest', 
    'api-key': myapikey, 
    'page-size': '10',
#     'q' : 'privacy' # All stories about privacy
#     'q' : 'privacy%20AND%20whatsapp%20AND%20signal' # privacy stories mentioning WhatsApp and Signal 
    'q' : 'privacy%20AND%20whatsapp%20AND%20signal%20AND%20NOT%20(AI%20OR%20%22Artificial%20Intelligence%22)%20'
    # privacy AND whatsapp AND signal AND NOT (AI OR "Artificial Intelligence"): stories not mentioning AI
}

# https://content.guardianapis.com/search?from-date=2020-01-01&to-date=2021-02-03&order-by=newest&page=1&q=%22Elon%20Musk%22%20AND%20bitcoin%20AND%20NOT%20Tesla&api-key=300efc2b-aa7f-4cac-b4ff-eff86a248059
    

# Construct the URL
base_url = 'http://content.guardianapis.com/search'
url = '{}?{}'.format(
    base_url, 
    '&'.join(["{}={}".format(kk, vv) for kk, vv in args.items()])
)

# Make the request and extract the source
req = requests.get(url) 
src = req.text

print('Number of byes received:', len(src))

# Parsing JSON
response = json.loads(src)['response']
print('The following are available:\n ', sorted(response.keys()))

# Verifying the status code
assert response['status'] == 'ok'

# Listing the results
print(json.dumps(response['results'][0], indent=2, sort_keys=True))

# Response statistics
print('Total news stories:', response['total']) 
print('Pages:', response['pages'])
print('Page size:', response['pageSize'])

Number of byes received: 6375
The following are available:
  ['currentPage', 'orderBy', 'pageSize', 'pages', 'results', 'startIndex', 'status', 'total', 'userTier']
{
  "apiUrl": "https://content.guardianapis.com/technology/2021/jan/26/uk-regulator-to-write-to-whatsapp-over-facebook-data-sharing",
  "id": "technology/2021/jan/26/uk-regulator-to-write-to-whatsapp-over-facebook-data-sharing",
  "isHosted": false,
  "pillarId": "pillar/news",
  "pillarName": "News",
  "sectionId": "technology",
  "sectionName": "Technology",
  "type": "article",
  "webPublicationDate": "2021-01-26T16:38:32Z",
  "webTitle": "UK regulator to write to WhatsApp over Facebook data sharing",
  "webUrl": "https://www.theguardian.com/technology/2021/jan/26/uk-regulator-to-write-to-whatsapp-over-facebook-data-sharing"
}
Total news stories: 32
Pages: 4
Page size: 10


In [15]:
# This cell is part of the initial iPython notebook that was shared.

for result in response['results']: 
    print(result['webUrl'][:70], result['webTitle'][:20])

https://www.theguardian.com/technology/2021/jan/26/uk-regulator-to-wri UK regulator to writ
https://www.theguardian.com/technology/2021/jan/24/whatsapp-loses-mill WhatsApp loses milli
https://www.theguardian.com/technology/2021/jan/24/is-it-time-to-leave Is it time to leave 
https://www.theguardian.com/technology/2021/jan/16/should-you-keep-usi Should you keep usin
https://www.theguardian.com/technology/askjack/2018/jun/28/can-my-empl Can my employer read
https://www.theguardian.com/technology/2018/mar/31/youve-decided-to-de You’ve decided to de
https://www.theguardian.com/technology/2018/feb/05/whatsapp-deleting-y I deleted WhatsApp f
https://www.theguardian.com/technology/2017/jul/14/forcing-facebook-go Australia's plan to 
https://www.theguardian.com/technology/2017/jun/19/eu-outlaw-backdoors EU seeks to outlaw '
https://www.theguardian.com/technology/2017/jun/12/george-brandiss-sal George Brandis's sal


## Task 8. Request Specific Content from the API

Fetch the ith result from the list obtained from on the search query formed in Task 7. 

Identify the id of the ith result and fetch the headline and body text of the news story.  

### Solution 
#### 1. Fetching the ith result

Let's now request a specific piece of content from the API. 

We select the ith result from the above response and get its ```apiUrl``` and ```id```:

In [16]:
i = 0
api_url = response['results'][i]['apiUrl']
api_id = response['results'][i]['id']

print(api_url)
print(api_id)

https://content.guardianapis.com/technology/2021/jan/26/uk-regulator-to-write-to-whatsapp-over-facebook-data-sharing
technology/2021/jan/26/uk-regulator-to-write-to-whatsapp-over-facebook-data-sharing


#### 2. Fetching the headline and body text of the news story with id 

We then use the ```id``` to contstruct a search url string to request this piece of content from the API.

(Note that you need to include the ```api-key``` in the search. You also need to specify if you want to include data fields other than the article metadata e.g. ```body``` and ```headline``` are included in the example below.)

In [17]:
base_url = "https://content.guardianapis.com/search?"
search_string = "ids=%s&api-key=%s&show-fields=headline,body" %(api_id, myapikey)

url = base_url + search_string
print(url)

https://content.guardianapis.com/search?ids=technology/2021/jan/26/uk-regulator-to-write-to-whatsapp-over-facebook-data-sharing&api-key=300efc2b-aa7f-4cac-b4ff-eff86a248059&show-fields=headline,body


In [18]:
req = requests.get(url) 
src = req.text

In [19]:
response = json.loads(src)['response']
assert response['status'] == 'ok'

In [20]:
print(response['results'][0]['fields']['headline'])

UK regulator to write to WhatsApp over Facebook data sharing


In [21]:
body = response['results'][0]['fields']['body']
print(body)

<p>The UK’s data regulator is writing to WhatsApp to demand that the chat app does not hand user data to Facebook, as millions worldwide continue to sign up for alternatives such as Signal and Telegram to avoid forthcoming changes to its terms of service.</p> <p>Elizabeth Denham, the information commissioner, told a parliamentary committee that in 2017, WhatsApp had committed not to hand any user information over to Facebook until it could prove that doing so respected GDPR.</p> <p>But, she said, that agreement was enforced by the Irish data protection authority until the Brexit transition period ended on 1 January. Now that Britain is fully outside the EU, ensuring that those promises are being kept falls to the Information Commissioner’s Office.</p> <p>“The change in the terms of service, and the requirement of users to share information with Facebook, does not apply to UK users or to users in the EU,” Denham told the digital, culture, media and sport sub-committee on online harms an

#### 3. Simple Text Processing: Count Word Frequencies and Store in a Data Frame

We can now do some simple text processing on the article text. e.g. count the word frequnecies:

In [22]:
words = body.replace('<p>','').replace('</p>','').split()
print(len(words))
unique_words = list(set(words))
print(len(unique_words))
#count_dictionary = {word: count for word, count in zip(words, [words.count(w) for w in words])}
count_dictionary = {'word': unique_words, 'count': [words.count(w) for w in unique_words]}

591
338


In [23]:
import pandas as pd

In [24]:
df = pd.DataFrame(count_dictionary)
df.sort_values(by='count', ascending=False)

# We could export the data frame in a CSV and observe the complete output
# df.to_csv('term-frequency.csv')

,word,count
18,the,43
162,to,24
27,and,14
224,of,12
6,that,12
...,...,...
132,Information,1
130,media,1
129,days,1
128,users.”,1


We now have a dataframe with word occurence frequency in the article. Try importing the data frame to a CSV.

We notice because of punctuation marks some words appear more than once.  For instance, ```UK``` also occurs as ```UK.``` and ```Signal``` also occurs as ```Signal,``` and ```Signal,"```. 

One option to fix this would be to strip out the punctuation using Python string manipulation. 

You could also use regular expressions to remove the punctuations. 

Below is an imperfect solution using regular expressions. You could probably find a better solution. 

In [25]:
import re  ## imports the regular expression library
words_wo_punctuation = re.sub(r'[^\w\s]','',body.replace('<p>','').replace('</p>','')).split()  

Note that the regex ```r'[^\w\s]'``` substitutes anything in ```body``` that is not a word ```\w``` or and blank space ```\s``` with the empty string ```''```.

In [26]:
unique_words = list(set(words_wo_punctuation))
print(len(unique_words))
count_dictionary = {'word': unique_words, 'count': [words_wo_punctuation.count(w) for w in unique_words]}

301


In [27]:
df = pd.DataFrame(count_dictionary)
df.sort_values(by='count', ascending=False)

# We could export the data frame in a CSV and observe the complete output
# df.to_csv('term-frequency-regex.csv')
# Open the CSV in a text editor or a spread sheet and analyse the output

,word,count
59,the,43
275,to,24
107,and,15
86,WhatsApp,13
20,that,12
...,...,...
130,itself,1
131,prevents,1
132,sign,1
134,office,1


This Python Regular Expression cheatsheet (Courtesy: Laura Gemmel) is useful: 
https://www.dataquest.io/wp-content/uploads/2019/03/python-regular-expressions-cheat-sheet.pdf

Use the cheat sheet to create a better regular expression and filter the body text. 
Post your solutions on the BB forum.